In [1]:
print('Saad')

Saad


In [ ]:
import os
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings,ChatHuggingFace,HuggingFaceEndpoint
from langchain_community.vectorstores import Chroma
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from dotenv import load_dotenv
load_dotenv()


True

In [ ]:
os.environ['HUGGINGFACEHUB_API_TOKEN']=''

In [3]:
%pwd

'd:\\ChatBotProjects\\DecoderBot\\research'

In [4]:
!mkdir sample_repo

In [ ]:

repo_path = "MedicalChatBot"
Repo.clone_from('https://github.com/M-SAAD-BIN-MAZHAR/MedicalChatBot',to_path=repo_path)


<git.repo.base.Repo 'd:\\ChatBotProjects\\DecoderBot\\research\\MedicalChatBot\\.git'>

In [6]:
loader=GenericLoader.from_filesystem(repo_path,
                                     glob='**/*.py',
                                     suffixes=['.py'],
                                     parser=LanguageParser(language=Language.PYTHON,parser_threshold=500))

In [8]:
documents=loader.load()
len(documents)

7

In [9]:
documents_splitter=RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,chunk_size=2000,chunk_overlap=200)

In [11]:
texts=documents_splitter.split_documents(documents)

In [12]:
len(texts)

6

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings,HuggingFaceEndpoint
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
llm=HuggingFaceEndpoint(
    repo_id='mistralai/Mistral-7B-Instruct-v0.3',
    task='text-generation'
)
model=ChatHuggingFace(llm=llm)

In [25]:
vectordb=Chroma.from_documents(texts,embedding=embeddings,persist_directory='./data')
vectordb.persist()

In [26]:
memory=ConversationSummaryMemory(llm=model,memory_key='chat_history',return_messages=True)

In [27]:
qa=ConversationalRetrievalChain.from_llm(model,retriever=vectordb.as_retriever(search_kwargs={'k':3}) ,memory=memory)

In [28]:
question='What is text_spliiter method'
result=qa(question)
print(result['answer'])

 The `text_split` method is a function that splits a given text into smaller chunks. It uses the `RecursiveCharacterTextSplitter` class from the `langchain.text_splitter` module. The method takes the text to be split as an argument and returns a list of smaller text chunks, each with a specified chunk size and overlap. The chunk size is set to 500 characters, and the overlap is set to 20 characters. This function is used to process large texts, such as those loaded from PDF files, into manageable chunks for further processing.
